# **Import libraries **

In [ ]:
!pip install transformers==2.11


     |████████████████████████████████| 674 kB 5.2 MB/s 
     |████████████████████████████████| 5.6 MB 39.1 MB/s 
     |████████████████████████████████| 895 kB 47.1 MB/s 
     |████████████████████████████████| 1.2 MB 44.5 MB/s 


In [ ]:
import numpy as np
import pandas as pd
from transformers import *
import torch
import tensorflow as tf
import pickle
import math as mt
from transformers import BertModel, TFBertModel

# **Import Data**

In [ ]:
!pip install datasets

     |████████████████████████████████| 312 kB 5.2 MB/s 
     |████████████████████████████████| 1.1 MB 36.1 MB/s 
     |████████████████████████████████| 67 kB 5.5 MB/s 
     |████████████████████████████████| 243 kB 51.2 MB/s 
     |████████████████████████████████| 133 kB 29.5 MB/s 
     |████████████████████████████████| 144 kB 47.4 MB/s 
     |████████████████████████████████| 94 kB 3.0 MB/s 
     |████████████████████████████████| 271 kB 50.4 MB/s 


In [ ]:
from datasets import load_dataset, load_metric
dataset = load_dataset("glue",  "stsb")

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/803k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/stsb/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 5749
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1379
    })
})

In [ ]:
df_traindata = dataset["train"]
df_train = pd.DataFrame(df_traindata)
#pd.read_csv('sts-train.tsv',sep='\t', error_bad_lines=False,encoding='utf-8')
df_devdata = dataset["validation"]
# pd.read_csv('sts-dev.tsv',sep='\t', error_bad_lines=False,encoding='utf-8')
df_dev = pd.DataFrame(df_devdata)
df_testdata = dataset["test"]
df_test = pd.DataFrame(df_testdata)

In [ ]:
df_dev

,sentence1,sentence2,label,idx
0,A man with a hard hat is dancing.,A man wearing a hard hat is dancing.,5.00,0
1,A young child is riding a horse.,A child is riding a horse.,4.75,1
2,A man is feeding a mouse to a snake.,The man is feeding a mouse to the snake.,5.00,2
3,A woman is playing the guitar.,A man is playing guitar.,2.40,3
4,A woman is playing the flute.,A man is playing a flute.,2.75,4
...,...,...,...,...
1495,Scientists prove there is water on Mars,Has Nasa discovered water on Mars?,2.00,1495
1496,Pranab stresses need to strive for peace by na...,WTO: India regrets action of developed nations,0.00,1496
1497,Volkswagen skids into red in wake of pollution...,"Volkswagen's ""gesture of goodwill"" to diesel o...",2.00,1497
1498,Obama is right: Africa deserves better leadership,Obama waiting for midterm to name attorney gen...,0.00,1498


In [ ]:
df_train = df_train.dropna()
df_dev = df_dev.dropna()
#df_test = df_test.dropna()

In [ ]:
df_dev

,sentence1,sentence2,label,idx
0,A man with a hard hat is dancing.,A man wearing a hard hat is dancing.,5.00,0
1,A young child is riding a horse.,A child is riding a horse.,4.75,1
2,A man is feeding a mouse to a snake.,The man is feeding a mouse to the snake.,5.00,2
3,A woman is playing the guitar.,A man is playing guitar.,2.40,3
4,A woman is playing the flute.,A man is playing a flute.,2.75,4
...,...,...,...,...
1495,Scientists prove there is water on Mars,Has Nasa discovered water on Mars?,2.00,1495
1496,Pranab stresses need to strive for peace by na...,WTO: India regrets action of developed nations,0.00,1496
1497,Volkswagen skids into red in wake of pollution...,"Volkswagen's ""gesture of goodwill"" to diesel o...",2.00,1497
1498,Obama is right: Africa deserves better leadership,Obama waiting for midterm to name attorney gen...,0.00,1498


# **Bert Config**

In [ ]:
config = BertConfig(dropout=0.2, attention_dropout=0.2, )
config.output_hidden_states = True

# **RoBert Tokenizer**

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True, add_special_tokens=True)
 
# config = RobertaConfig()
# config.output_hidden_states = True
config = RobertaConfig.from_pretrained("roberta-base")
config.output_hidden_states = True
transformer_model = TFRobertaModel.from_pretrained('roberta-base', config=config)
transformer_model.trainable = True
transformer_model.output_hidden_states = True

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/657M [00:00<?, ?B/s]

# **Tokenize Function**

In [ ]:
def tokenize(df, tokenizer, indicator):
    input_ids, input_masks, input_segments, input_offsets = [],[],[],[]
    if indicator == 1:
      tmp = 'sentence1'
    elif indicator == 2:
      tmp = 'sentence2'
    for i in range(len(df)):
        #print(i)
        inputs = tokenizer.encode_plus(df[tmp].iloc[i], truncation=True ,
                                       add_special_tokens=True, max_length=32, pad_to_max_length=True, 
                                             return_attention_mask=True, return_token_type_ids=True)
        #print(i)
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
        input_segments.append(inputs['token_type_ids'])

        # inputs = tokenizer.encode_plus(df['sentence2'].iloc[i], add_special_tokens=True, 
        #                                      return_attention_mask=True, return_token_type_ids=True)
        # #print(i)
        # input_ids.append(inputs['input_ids'])
        # input_masks.append(inputs['attention_mask'])
        # input_segments.append(inputs['token_type_ids'])  
        #input_offsets.append(inputs[''])       
        
    #return input_ids, input_masks, input_segments
    return tf.cast(input_ids,tf.int32),tf.cast(input_masks,tf.int32), tf.cast(input_segments,tf.int32) 

# **Tokenize Sentence1 and Sentence2 for train, test and dev dataframe**

In [ ]:
input_ids1, input_masks1,input_segments1 = tokenize(df_train, tokenizer, 1)

In [ ]:
dev_ids1, dev_masks1,dev_segments1 = tokenize(df_dev, tokenizer, 1)

In [ ]:
test_ids1, test_masks1,test_segments1 = tokenize(df_test, tokenizer, 1)

In [ ]:
input_ids2, input_masks2,input_segments2 = tokenize(df_train, tokenizer, 2)
dev_ids2, dev_masks2,dev_segments2 = tokenize(df_dev, tokenizer, 2)
test_ids2, test_masks2,test_segments2 = tokenize(df_test, tokenizer, 2)

# **Import libraries again :)**

In [ ]:
import tensorflow.keras.backend as K
from time import time
import numpy as np
import tensorflow.keras.backend as K
#from keras.engine.topology import Layer, InputSpec
from tensorflow.keras.layers import Dense, Input,Lambda, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import callbacks
from tensorflow.keras.initializers import VarianceScaling
#from sklearn.cluster import KMeans
import tensorflow_hub as hub
import os
#from tensorflow.contrib.factorization import KMeans
#import metrics

# **Define testModel class (RoBERTa)**



In [ ]:
class testmodel(tf.keras.Model):
    """Combines the encoder and decoder into an end-to-end model for training."""

    def __init__(
        self,
        name="ISObn",
        **kwargs
    ):
        super(testmodel, self).__init__(name=name, **kwargs)

        self.config = config        
        transformer_model = TFRobertaModel.from_pretrained('roberta-base', config = self.config)
        transformer_model.trainable = True
        for layer in transformer_model.layers:
          self.ROBERTA = layer

        transformer_model1 = TFRobertaModel.from_pretrained('roberta-base', config = self.config)
        transformer_model1.trainable = True
        for layer in transformer_model1.layers:
          self.ROBERTA1 = layer

        self.representation_l = []
        for i in range(13):
          tmp = []
          self.representation_l.append(tmp)


        self.representation_r = []

        for i in range(13):
          tmp = []
          self.representation_r.append(tmp)
        # self.representations_0 =[]
        # self.representations_1 =[]
        # self.representations_2 =[]
        # self.representations_3 =[]
        # self.representations_4 =[]
        # self.representations_5 =[]
        # self.representations_6 =[]
        # self.representations_7 =[]
        # self.representations_8 =[]
        # self.representations_9 =[]
        # self.representations_10 =[]
        # self.representations_11 =[]
        # self.representations_12 =[]
        # self.representations_0_r =[]
        # self.representations_1_r =[]
        # self.representations_2_r =[]
        # self.representations_3_r =[]
        # self.representations_4_r =[]
        # self.representations_5_r =[]
        # self.representations_6_r =[]
        # self.representations_7_r =[]
        # self.representations_8_r =[]
        # self.representations_9_r =[]
        # self.representations_10_r =[]
        # self.representations_11_r =[]
        # self.representations_12_r =[]
        self.left_rep =[]
        self.right_rep =[]

        #self.dense = tf.keras.layers.Dense(1, activation='sigmoid')



        


    def call(self, inputs ):

        x = self.ROBERTA(inputs[0], inputs[1],inputs[2])
        flag = inputs[3]
        x2 = self.ROBERTA1(inputs[4], inputs[5],inputs[6])

        left_sent = tf.math.reduce_mean(x[0], axis = 1)
        right_sent = tf.math.reduce_mean(x2[0], axis = 1)

        out = -1 * tf.keras.losses.cosine_similarity(left_sent, right_sent)
        if flag[0] == 1:
              # for i in range(13):
              #     self.representation_l[i].append(x[2][i])

              # self.representations_0.append(x[2][0])  
              # self.representations_1.append(x[2][1])
              # self.representations_2.append(x[2][2])
              # self.representations_3.append(x[2][3])
              # self.representations_4.append(x[2][4])
              # self.representations_5.append(x[2][5])
              # self.representations_6.append(x[2][6])
              # self.representations_7.append(x[2][7])
              # self.representations_8.append(x[2][8])
              # self.representations_9.append(x[2][9])
              # self.representations_10.append(x[2][10])
              # self.representations_11.append(x[2][11])
              # self.representations_12.append(x[2][12])
              self.left_rep.append(x[0])

              # for i in range(13):
              #     self.representation_r[i].append(x2[2][i])

              # self.representations_0_r.append(x2[2][0])  
              # self.representations_1_r.append(x2[2][1])
              # self.representations_2_r.append(x2[2][2])
              # self.representations_3_r.append(x2[2][3])
              # self.representations_4_r.append(x2[2][4])
              # self.representations_5_r.append(x2[2][5])
              # self.representations_6_r.append(x2[2][6])
              # self.representations_7_r.append(x2[2][7])
              # self.representations_8_r.append(x2[2][8])
              # self.representations_9_r.append(x2[2][9])
              # self.representations_10_r.append(x2[2][10])
              # self.representations_11_r.append(x2[2][11])
              # self.representations_12_r.append(x2[2][12])
              self.right_rep.append(x2[0])


        return out

# **Define SpearMan function**

In [ ]:
from scipy.stats import spearmanr
def spearman(labels, preds):
    return (tf.py_function(spearmanr, [tf.cast(labels, tf.float32), 
                           tf.cast(preds, tf.float32)], Tout = tf.float32))


# **Build a model**

In [ ]:
cce = tf.keras.losses.CategoricalCrossentropy()
model = testmodel()
optimizer = tf.keras.optimizers.Adam(7e-5)
tf.config.run_functions_eagerly(True)
model.compile(optimizer=optimizer, loss ='mean_squared_error', metrics=[spearman ])

In [ ]:
x =[0]
x = np.zeros(len(df_train))
y = np.zeros(len(df_dev))
train_inp = [input_ids1, input_masks1, input_segments1, x,  input_ids2, input_masks2, input_segments2]
dev_inp = [dev_ids1, dev_masks1, dev_segments1, y, dev_ids2, dev_masks2, dev_segments2]

In [ ]:
list(df_train['label'])

[5.0,
 3.799999952316284,
 3.799999952316284,
 2.5999999046325684,
 4.25,
 4.25,
 0.5,
 1.600000023841858,
 2.200000047683716,
 5.0,
 4.199999809265137,
 4.599999904632568,
 3.867000102996826,
 4.666999816894531,
 1.6670000553131104,
 3.75,
 5.0,
 0.5,
 3.799999952316284,
 5.0,
 3.200000047683716,
 2.799999952316284,
 4.599999904632568,
 3.0,
 5.0,
 4.800000190734863,
 5.0,
 4.199999809265137,
 4.199999809265137,
 4.0,
 4.0,
 4.908999919891357,
 3.0,
 2.4000000953674316,
 4.199999809265137,
 3.4000000953674316,
 5.0,
 3.75,
 2.75,
 5.0,
 4.0,
 3.5999999046325684,
 1.600000023841858,
 1.75,
 5.0,
 1.0,
 1.0,
 2.375,
 3.799999952316284,
 3.200000047683716,
 3.200000047683716,
 4.400000095367432,
 3.75,
 4.75,
 3.200000047683716,
 1.555999994277954,
 3.937999963760376,
 5.0,
 5.0,
 4.0,
 1.600000023841858,
 4.75,
 3.5,
 1.399999976158142,
 1.399999976158142,
 4.0,
 5.0,
 3.8329999446868896,
 0.6000000238418579,
 2.9170000553131104,
 4.199999809265137,
 2.0,
 2.5999999046325684,
 1.6000000

# **Pre-Trained Model**

In [ ]:
x =[0]
x = np.zeros(len(df_train))
y = np.ones(len(df_dev))
train_inp = [input_ids1, input_masks1, input_segments1, x,  input_ids2, input_masks2, input_segments2]
dev_inp = [dev_ids1, dev_masks1, dev_segments1, y, dev_ids2, dev_masks2, dev_segments2]

# **Evaluate the model**

In [ ]:
model.evaluate(dev_inp, df_dev['label']/5)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4527: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


47/47 [==============================] - 23s 487ms/step - loss: 0.3332 - spearman: 0.2601


[0.33317121863365173, 0.26008421182632446]

# **Import libraries again and again :))**

In [ ]:
import numpy as np
import pandas as pd
from transformers import *
import torch
import tensorflow as tf
import pickle
import scipy as sc
import math as mt
from scipy import cluster as clst
from sklearn.metrics.pairwise import cosine_similarity
from numpy import linalg as LA
from sklearn.decomposition import PCA
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.mixture import GaussianMixture
import nltk
import random

# **Representation left sentence and right sentence of model**

In [ ]:
all_rep_left = []
for i in range(len(model.left_rep)):
  for j in range(model.left_rep[i].shape[0]):
    for k in range(32):
      all_rep_left.append(np.reshape(model.left_rep[i][j][k],(768)))

In [ ]:
all_rep_right = []
for i in range(len(model.right_rep)):
  for j in range(model.right_rep[i].shape[0]):
    for k in range(32):
      all_rep_right.append(np.reshape(model.right_rep[i][j][k],(768)))

In [ ]:
train_data_left = np.reshape(all_rep_left, (-1, 32, 768))

In [ ]:
train_data_right = np.reshape(all_rep_right, (-1, 32, 768))

In [ ]:
#getting the representation embedding for the left sentence by averaging over all word emebddings in a sentence
feature_left = np.asarray(train_data_left).reshape(-1,32, 768)
feature_left = feature_left.mean(axis=1)
feature_left.shape

(1500, 768)

In [ ]:
#getting the representation embedding for the right sentence
feature_right = np.asarray(train_data_right).reshape(-1,32, 768)
feature_right = feature_right.mean(axis=1)
feature_right.shape

(1500, 768)

# **Calculating cosine similarity **

In [ ]:
#calculating cosing similarity between two sentences
score = []
l = 0
for i in range(len(df_dev)):
 score.append(cosine_similarity(np.reshape(feature_left[i], (1, 768)), np.reshape(feature_right[i], (1, 768)))[0][0])


In [ ]:
spearman(score, df_dev['label']/5)

<tf.Tensor: shape=(), dtype=float32, numpy=0.3422646>

In [ ]:
import math as mt

In [ ]:
all_rep = all_rep_right + all_rep_left
from sklearn.decomposition import PCA as pca

x = pca(3)
new_rep = x.fit_transform(all_rep)
new_rep.shape

(96000, 3)

In [ ]:
cars = {'x': new_rep[:,0],
        'y': new_rep[:,1],
        'z': new_rep[:,2],
        }

df = pd.DataFrame(cars, columns = ['x', 'y', 'z'])


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')
ax.scatter(df['x'],df['y'], df['z'])

In [ ]:
r = range(149)

In [ ]:
representation_list = all_rep_left + all_rep_right

In [ ]:
k=1#number of clusters
centroid, label=clst.vq.kmeans2(representation_list, k, minit='points', missing='warn', check_finite=True)

In [ ]:
cluster_mean=[]
for i in range(max(label)+1):
  sum=np.zeros([1,768]);
  for j in np.nonzero(label == i)[0]:
    sum=np.add(sum, representation_list[j])
  cluster_mean.append(sum/len(label[label == i]))

In [ ]:
post_representation_list=[]
for i in range(len(representation_list)):
  post_representation_list.append((representation_list[i] - cluster_mean[label[i]]))

In [ ]:
cluster_representations={}
for i in range(k):
  cluster_representations.update({i:{}})
  for j in range(len(representation_list)):
    if (label[j]==i):
      cluster_representations[i].update({j:post_representation_list[j]})

In [ ]:
cluster_representations2=[]
for j in range(k):
  cluster_representations2.append([])
  for key, value in cluster_representations[j].items():
    cluster_representations2[j].append(value)

cluster_representations2=np.array(cluster_representations2)
len(cluster_representations2[0])


96000

In [ ]:
post_rep2=np.zeros((len(representation_list),768))
D =4
for i in range(k):
    x.fit(np.array(cluster_representations2[i]).reshape((-1,768)))
    component = np.reshape(x.components_, (-1, 768))
    for index in cluster_representations[i]:
        sum_vec = np.zeros((1, 768))
        for j in range(D):
              sum_vec = sum_vec + np.dot(cluster_representations[i][index], np.transpose(component)[:,-j].reshape((768,1)) ) * component[-j]
        post_rep2[index]=cluster_representations[i][index] - sum_vec

#####################################
"""

sum_index=0
j=0
final_features=[]
for i in range(len(representation)):
  final_features.append(np.mean(post_rep2[sum_index:sum_index+(len(representation[i]))],axis=0))
  j=j+1
  sum_index=sum_index+len(representation[i])

# #####################################

score = []
l = 0
for i in range(int(len(final_features)/2)):
  score.append(cosine_similarity(np.reshape(final_features[l], (1, 768)), np.reshape(final_features[l + 1], (1, 768)))[0][0])
  l = l + 2
  """

'\n\nsum_index=0\nj=0\nfinal_features=[]\nfor i in range(len(representation)):\n  final_features.append(np.mean(post_rep2[sum_index:sum_index+(len(representation[i]))],axis=0))\n  j=j+1\n  sum_index=sum_index+len(representation[i])\n\n# #####################################\n\nscore = []\nl = 0\nfor i in range(int(len(final_features)/2)):\n  score.append(cosine_similarity(np.reshape(final_features[l], (1, 768)), np.reshape(final_features[l + 1], (1, 768)))[0][0])\n  l = l + 2\n  '

In [ ]:
post_rep2.shape

(96000, 768)

In [ ]:
post_rep2 = list(post_rep2)
new_left = post_rep2[:int(len(post_rep2)/2)]
new_right = post_rep2[int(len(post_rep2)/2):]

In [ ]:
from sklearn.decomposition import PCA as pca
all_rep_new = new_left + new_right
x = pca(2)
new_rep = x.fit_transform(np.array(all_rep_new).reshape((-1,768)))
new_rep.shape

(96000, 2)

In [ ]:
from sklearn.decomposition import PCA as pca

x = pca(2)
new_rep = x.fit_transform(np.array(all_rep).reshape((-1,768)))
new_rep.shape

(96000, 2)

# **Define basic functions**

In [ ]:

def sper_corrcoef(targets, predictions):
    """Spearman correlation coefficient."""
    return 100 * sc.stats.spearmanr(targets, predictions)[0]


def mean_pooling(inp_representations, representation_dev):
    """ calculating sentence representations by averaging over the tokens."""

    sum_index=0
    sent_representations=[]
    for i in range(len(representation_dev)):
      sent_representations.append(np.mean(inp_representations[sum_index: sum_index + (len(representation_dev[i]))],axis=0))
      sum_index = sum_index + len(representation_dev[i])

    return sent_representations


def similarity(sent_rep):
    """ calculating cosine similarity between two sentences."""
  
    score = []
    l = 0
    for i in range(int(len(sent_rep)/2)):
        score.append(cosine_similarity(np.reshape(sent_rep[l], (1, 768)),
                                      np.reshape(sent_rep[l + 1], (1, 768)))[0][0])
        l = l + 2

    return score


def isotropy(representations):
    """Calculating isotropy of embedding space based on Eq.2
           arg:
              representations (n_samples, n_dimensions)
            """

    eig_values, eig_vectors = np.linalg.eig(np.matmul(np.transpose(representations),
                                                      representations))
    max_f = -mt.inf
    min_f =  mt.inf

    for i in range(eig_vectors.shape[1]):
        f = np.matmul(representations, np.expand_dims(eig_vectors[:, i], 1))
        f = np.sum(np.exp(f))

        min_f = min(min_f, f)
        max_f = max(max_f, f)

    isotropy = min_f / max_f

    return isotropy


# **Cluster-Based App function**

In [ ]:

def cluster_based(representations, n_cluster: int, n_pc: int):
  """ Improving Isotropy of input representations using cluster-based method
      Args: 
            inputs:
                  representations: 
                    input representations numpy array(n_samples, n_dimension)
                  n_cluster: 
                    the number of clusters
                  n_pc: 
                    the number of directions to be discarded
            output:
                  isotropic representations (n_samples, n_dimension)

            """


  centroid, label=clst.vq.kmeans2(representations, n_cluster, minit='points',
                                  missing='warn', check_finite=True)
  cluster_mean=[]
  for i in range(max(label)+1):
    sum=np.zeros([1,768]);
    for j in np.nonzero(label == i)[0]:
      sum=np.add(sum, representations[j])
    cluster_mean.append(sum/len(label[label == i]))

  zero_mean_representation=[]
  for i in range(len(representations)):
    zero_mean_representation.append((representations[i])-cluster_mean[label[i]])

  cluster_representations={}
  for i in range(n_cluster):
    cluster_representations.update({i:{}})
    for j in range(len(representations)):
      if (label[j]==i):
        cluster_representations[i].update({j:zero_mean_representation[j]})

  cluster_representations2=[]
  for j in range(n_cluster):
    cluster_representations2.append([])
    for key, value in cluster_representations[j].items():
      cluster_representations2[j].append(value)

  cluster_representations2=np.array(cluster_representations2)


  model=PCA()
  post_rep=np.zeros((representations.shape[0],representations.shape[1]))

  for i in range(n_cluster):
      model.fit(np.array(cluster_representations2[i]).reshape((-1,768)))
      component = np.reshape(model.components_, (-1, 768))

      for index in cluster_representations[i]:
        sum_vec = np.zeros((1, 768))

        for j in range(n_pc):
                sum_vec = sum_vec + np.dot(cluster_representations[i][index],
                          np.transpose(component)[:,j].reshape((768,1))) * component[j]
        
        post_rep[index]=cluster_representations[i][index] - sum_vec
  return post_rep


# **Getting representations**

In [ ]:
casing = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(casing, do_lower_case=True, add_special_tokens=True)

config = BertConfig(dropout=0.2, attention_dropout=0.2, )
config.output_hidden_states = True

model = TFBertModel.from_pretrained(casing, config = config)
model.trainable = False

In [ ]:

representation_dev = []
for i in range(len(df_dev)):
    print(i)
    #First sentence
    inputs = tokenizer.encode(df_dev['sentence1'].iloc[i], add_special_tokens=True)
    inputs = np.asarray(inputs, dtype='int32').reshape((1, -1))

    #getting the representation of the last layer
    output = model(inputs)[0]
    output = np.asarray(output).reshape((-1,768))

    #Removing CLS and SEP tokens
    idx = [0, len(output)-1]
    output = np.delete(output, idx, axis= 0)
    output = np.asarray(output).reshape((-1,768))

    representation_dev.append(output)

    #Second sentence
    inputs = tokenizer.encode(df_dev['sentence2'].iloc[i], add_special_tokens=True)
    inputs = np.asarray(inputs, dtype='int32').reshape((1, -1))

    output = model(inputs)[0]
    output = np.asarray(output).reshape((-1,768))

    #Removing CLS and SEP tokens
    idx = [0, len(output)-1]
    output = np.delete(output, idx, axis= 0)
    output = np.asarray(output).reshape((-1,768))

    representation_dev.append(output)

representation_list_dev=[]
for i in range(len(representation_dev)):
  for j in range(len(representation_dev[i])):
      representation_list_dev.append(representation_dev[i][j])



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

# **Zero-mean Function**

In [ ]:
## zero mean
def zero_mean(representations):
  rep_sum=0
  for i in range(len(representations)):
    rep_sum= rep_sum + representations[i]
  rep_mean= rep_sum/len(representations)
  zero_mean=[]
  for i in range(len(representations)):
    zero_mean.append(representations[i]-rep_mean)
    
  zero_mean_array=np.array(zero_mean)
  return zero_mean
    

  
    

# **Global app**

In [ ]:
  
def Global_app(representations, n_cluster: int, n_pc: int):
  """ Improving Isotropy of input representations using cluster-based method
      Args: 
            inputs:
                  representations: 
                    input representations numpy array(n_samples, n_dimension)
                  n_cluster: 
                    the number of clusters
                  n_pc: 
                    the number of directions to be discarded
            output:
                  isotropic representations (n_samples, n_dimension)

            """


  centroid, label=clst.vq.kmeans2(representations, n_cluster, minit='points',
                                  missing='warn', check_finite=True)
  cluster_mean=[]
  for i in range(max(label)+1):
    sum=np.zeros([1,768]);
    for j in np.nonzero(label == i)[0]:
      sum=np.add(sum, representations[j])
    cluster_mean.append(sum/len(label[label == i]))

  zero_mean_representation=[]
  for i in range(len(representations)):
    zero_mean_representation.append((representations[i])-cluster_mean[label[i]])

  cluster_representations={}
  for i in range(n_cluster):
    cluster_representations.update({i:{}})
    for j in range(len(representations)):
      if (label[j]==i):
        cluster_representations[i].update({j:zero_mean_representation[j]})

  cluster_representations2=[]
  for j in range(n_cluster):
    cluster_representations2.append([])
    for key, value in cluster_representations[j].items():
      cluster_representations2[j].append(value)

  cluster_representations2=np.array(cluster_representations2)


  model=PCA()
  post_rep=np.zeros((representations.shape[0],representations.shape[1]))

  for i in range(n_cluster):
      model.fit(np.array(cluster_representations2[i]).reshape((-1,768)))
      component = np.reshape(model.components_, (-1, 768))

      for index in cluster_representations[i]:
        sum_vec = np.zeros((1, 768))

        for j in range(n_pc):
                sum_vec = sum_vec + np.dot(cluster_representations[i][index],
                          np.transpose(component)[:,j].reshape((768,1))) * component[j]
        
        post_rep[index]=cluster_representations[i][index] - sum_vec
  return post_rep


# **Clustering + zero-mean**

In [ ]:
def cluster_zero(representations, n_cluster: int, n_pc: int):
  """ Improving Isotropy of input representations using cluster-based method
      Args: 
            inputs:
                  representations: 
                    input representations numpy array(n_samples, n_dimension)
                  n_cluster: 
                    the number of clusters
                  n_pc: 
                    the number of directions to be discarded
            output:
                  isotropic representations (n_samples, n_dimension)

            """


  centroid, label=clst.vq.kmeans2(representations, n_cluster, minit='points',
                                  missing='warn', check_finite=True)
  cluster_mean=[]
  for i in range(max(label)+1):
    sum=np.zeros([1,768]);
    for j in np.nonzero(label == i)[0]:
      sum=np.add(sum, representations[j])
    cluster_mean.append(sum/len(label[label == i]))

  zero_mean_representation=[]
  for i in range(len(representations)):
    zero_mean_representation.append((representations[i])-cluster_mean[label[i]])
  zero_mean_representation = np.array(zero_mean_representation)
  return zero_mean_representation


# **Making the representations isotorpic**

In [ ]:
def isotropy(representations):
    """Calculating isotropy of embedding space based on Eq.2
           arg:
              representations (n_samples, n_dimensions)
            """

    eig_values, eig_vectors = np.linalg.eig(np.matmul(np.transpose(representations),
                                                      representations))
    max_f = -mt.inf
    min_f =  mt.inf

    for i in range(eig_vectors.shape[1]):
        f = np.matmul(representations, np.expand_dims(eig_vectors[:, i], 1))
        f = np.sum(np.exp(f))

        min_f = min(min_f, f)
        max_f = max(max_f, f)

    isotropy = min_f / max_f

    return isotropy

# **Cluster-Based app on ROBERTA**

In [ ]:
n_cluster = 27
n_pc = 12
print(len(representation_list_dev))
isotropic_representations = cluster_based(np.asarray(representation_list_dev),
                                          n_cluster, n_pc)

# calculating sentence representations
sentence_rep = mean_pooling(isotropic_representations, representation_dev)

# predicting similarity scores
score = similarity(sentence_rep)


43605


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
#isotropic_representations

In [ ]:
# performance
print("Spearman Correlation: ",sper_corrcoef(df_dev['label'], score))

# isotropy of space
print("Isotropy: ", isotropy(isotropic_representations))

Spearman Correlation:  58.43633445565197
Isotropy:  0.91066827396211486


# **Base-line on ROBERTA**

In [ ]:
#pre trained
n_cluster = 27
n_pc = 12
print(len(representation_list_dev))
isotropic_representations = representation_list_dev
# calculating sentence representations
sentence_rep = mean_pooling(isotropic_representations, representation_dev)

# predicting similarity scores
score = similarity(sentence_rep)


42709


In [ ]:
# performance
print("Spearman Correlation: ",sper_corrcoef(df_dev['label'], score))
# isotropy of space
print("Isotropy: ", isotropy(representation_list_dev))

Spearman Correlation:  36.51478521358954
Isotropy:  2.957412e-6


# **Zero-Mean approach on ROBertA**

In [ ]:
z=zero_mean(representation_list_dev)

In [ ]:
n_cluster = 27
n_pc = 12
print(len(representation_list_dev))
isotropic_representations = zero_mean(representation_list_dev)
# calculating sentence representations
sentence_rep = mean_pooling(isotropic_representations, representation_dev)

# predicting similarity scores
score= similarity(sentence_rep)


43605


In [ ]:
# performance
print("Spearman Correlation: ",sper_corrcoef(df_dev['label'], score))

# isotropy of space
print("Isotropy: ", isotropy(z))


Spearman Correlation:  35.43633445565197
Isotropy:  7.950029137940146e-02


In [ ]:
Global_app(np.asarray(representation_list_dev) , 1 , 12 )

In [ ]:
cluster_zero(np.asarray(representation_list_dev) , 27 , 12 )




#Global_app_ROBERTA

In [ ]:
n_cluster = 1
n_pc = 12
print(len(representation_list_dev))
g = Global_app(np.asarray(representation_list_dev) , 1 , 12 )
isotropic_representations = g
# calculating sentence representations
sentence_rep = mean_pooling(isotropic_representations, representation_dev)

# predicting similarity scores
score = similarity(sentence_rep)

43605


In [ ]:
# performance
print("Spearman Correlation: ",sper_corrcoef(df_dev['label'], score))

# isotropy of space
print("Isotropy: ", isotropy(g))

Spearman Correlation:  63.43633445565197
Isotropy:  0.8227963480977477


#Cluster+zero_mean

In [ ]:
#pre trained
n_cluster = 27
n_pc = 12
print(len(representation_list_dev))
isotropic_representations = cluster_zero(np.asarray(representation_list_dev),
                                          n_cluster, n_pc)

# calculating sentence representations
sentence_rep = mean_pooling(isotropic_representations, representation_dev)

# predicting similarity scores
score = similarity(sentence_rep)


42709


In [ ]:
# performance
print("Spearman Correlation: ",sper_corrcoef(df_dev['label'], score))

# isotropy of space
print("Isotropy: ", isotropy(isotropic_representations))

Spearman Correlation:  62.58412369847526
Isotropy:  0.70451697425305
